In [ ]:
%cd ..

/content/drive/MyDrive/gan_detect_iris


In [ ]:
%pwd

'/content/drive/My Drive/gan_detect_iris'

In [ ]:
%cd '/content/drive/MyDrive/gan_detect_iris'

/content/drive/MyDrive/gan_detect_iris


In [ ]:
%ls

1000_Faces/                                     gfg_dummy_pic_right.png
100_Faces/                                      iris_left_resize.png
250_Faces/                                      iris_right_resize.png
250_Fake_Faces/                                 main.py
250_Real_Faces.zip                              out.csv
500_Faces/                                      outputs/
buffalo_generated_visual_content_detection.log  out.zip
crop_cornea.py                                  __pycache__/
crop_eyes.py                                    README.md
crop_highlights.py                              RealFace/
crop_iris.py                                    Real_Face_Final/
data/                                           requirements.txt
gfg_dummy_pic_left.png                          shape_predictor/


In [ ]:
%cd 500_Faces

/content/drive/MyDrive/gan_detect_iris/500_Faces


In [ ]:
%mkdir 500_Real_Face

In [ ]:
%cd 500_Real_Face

In [ ]:
%cd 500_real_face_extract

/content/drive/MyDrive/gan_detect_iris/500_Faces/500_real_face_extract


In [ ]:
%cd /content/drive/MyDrive/gan_detect_iris/1000_Faces/100_real

/content/drive/MyDrive/gan_detect_iris/1000_Faces/100_real


In [ ]:

!unzip 2nd_100_Male_Face.zip

In [ ]:
!ls -1 | wc -l

100


In [ ]:

#!unzip '/content/100_Male_Face.zip'  -d '/content/drive/MyDrive/gan_detect_iris/1000_Faces/100_real'


Archive:  /content/100_Male_Face.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/100_Male_Face.zip or
        /content/100_Male_Face.zip.zip, and cannot find /content/100_Male_Face.zip.ZIP, period.


In [ ]:

!unzip '/content/Male_Face.zip'  -d '/content/drive/MyDrive/gan_detect_iris/1000_Faces/Real'


In [ ]:
%pwd

'/content/drive/MyDrive/gan_detect_iris'

In [ ]:
%cd ..

/


**Input directory paths**

In [ ]:

Real_img='/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Real_Face'
Fake_img='/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face'


In [ ]:
import numpy as np
import argparse
import dlib
import logging
import cv2
import shutil
import sys
import os
import math
import argparse
import glob
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from crop_eyes import crop_eye, drawPoints, eye_detection
from crop_cornea import cornea_convex_hull
from crop_iris import  segment_iris
from crop_highlights import process_aligned_image

logging.basicConfig(level=logging.INFO,filename='buffalo_generated_visual_content_detection.log')
logger = logging.getLogger("buffalo_generated_visual_content_detection")

import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

def Give_me_Landmarks(f):
  img = imageio.imread(f)
  landmarks= extract_face_landmarks(img)
  res=[]
  res = landmarks.flatten().tolist()
  return res


def Detect(img):
  #Crop Eye
    left_eye_image, right_eye_image, new_eyes_position_list, number_face, double_eye_img, double_eye_position_difference_list = eye_detection(img, './shape_predictor/shape_predictor_68_face_landmarks.dat')
    #print("CHECK!!Crop Eye 1")
    #Crop Cornea
    left_cornea, right_cornea, left_cornea_matrix, right_cornea_matrix \
            = cornea_convex_hull(left_eye_image, right_eye_image, new_eyes_position_list)
    #print("check!! Crop Cornea 2")
    #Crop iris
    img_left, iris_left, l_iris, l_iris_center, l_radius, l_eye_center, l_highlights, l_num_refl, l_valid \
            = segment_iris(left_eye_image, left_cornea_matrix.astype(bool), 4.5,
                           2.0)  # 'left'
    img_right, iris_right, r_iris, r_iris_center, r_radius, r_eye_center, r_highlights, r_num_refl, r_valid \
            = segment_iris(right_eye_image, right_cornea_matrix.astype(bool),4.5,
                           2.0)  # 'right'

    if l_num_refl==0 and r_num_refl==0:
            return False
    #print("Check!! Crop iris 3")

    # 5. Draw circles on double eyes
    double_eye_img_ori = double_eye_img.copy()
    new_left_eye = l_iris_center + double_eye_position_difference_list[0]
    new_right_eye = r_iris_center + double_eye_position_difference_list[1]
    cv2.circle(double_eye_img, (new_left_eye[0], new_left_eye[1]), l_radius, (0, 0, 255), 2)  # left
    cv2.circle(double_eye_img, (new_right_eye[0], new_right_eye[1]), r_radius, (0, 0, 255), 2)  # right

    #print("check Draw circles on double eyes 4")
    #Crop Highlights
    iris_left_resize, iris_right_resize, left_recolor, right_recolor, \
        left_recolor_resize, right_recolor_resize, IOU_score, double_eye_img_modified \
            = process_aligned_image(iris_left, iris_right, l_iris, r_iris, l_highlights, r_highlights, left_eye_image,
                                    right_eye_image,
                                    double_eye_img, double_eye_position_difference_list, reduce=True,
                                    reduce_size=2, threshold_scale_left=1.2,
                                    threshold_scale_right=1.2)



    #print('IOU_score:', IOU_score)
    return IOU_score

def main(directory):
  x=[]
  p=0
  q=0
  #directory = '/content/drive/MyDrive/gan_detect_iris/data'
  #directory = '/content/drive/MyDrive/StyleGAN3_images'
  #directory = '/content/drive/Othercomputers/My Laptop/Desktop/1000_Fake_Faces'
  for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
      print(f)
      score = Detect(f)
      b=Give_me_Landmarks(f)
      b.insert(len(b),score)
      #print(b)
      x.append(b)
      q=q+1
      #print(q)
      if(not score):
        p=p+1;
      #x.append(score)
      #arr = np.array(x)
      #print(arr)
      #df = pd.DataFrame(data = arr[0:, 0:],index = ['Row_' + str(i + 1) for i in range(arr.shape[0])],columns = ['Column_' + str(i + 1) for i in range(arr.shape[1])])
  #print(x)
  #print(p)
  return x

  """x=[]
  img='./data/seed000003.png'
  score=Detect(img)
  b=Give_me_Landmarks(f)
  b.insert(len(b),score)
  print(b)
  x.append(b)
  return x
  """



if __name__ == "__main__":
  try:
    """
    #Real Img
    directory = Real_img
    x=main(directory)
    arr = np.array(x)
    df1 = pd.DataFrame(data = arr[0:, 0:],index = ['Row_R' + str(i + 1) for i in range(arr.shape[0])],columns = ['Column_' + str(i + 1) for i in range(arr.shape[1])])

    #Fake image
    directory = Fake_img
    x=main(directory)
    arr = np.array(x)
    df2 = pd.DataFrame(data = arr[0:, 0:],index = ['Row_F' + str(i + 1) for i in range(arr.shape[0])],columns = ['Column_' + str(i + 1) for i in range(arr.shape[1])])
    """

    directory = Fake_img
    x=main(directory)
    arr = np.array(x)
    #print(arr)
    df1 = pd.DataFrame(data = arr[0:, 0:],index = ['Row_R' + str(i + 1) for i in range(arr.shape[0])],columns = ['Column_' + str(i + 1) for i in range(arr.shape[1])])


    directory = Real_img
    x=main(directory)
    #print(x)
    arr = np.array(x)
    #print(arr)
    df2 = pd.DataFrame(data = arr[0:, 0:],index = ['Row_F' + str(i + 1) for i in range(arr.shape[0])],columns = ['Column_' + str(i + 1) for i in range(arr.shape[1])])

  except:
        logger.error('Get IoU failed.')




/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/0.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/1.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/2.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/3.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/4.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/5.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/6.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/7.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/8.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/10.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/9.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/11.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/12.jpg
/content/drive/MyDrive/gan_detect_iris/100_Faces/100_Fake_Face/13.jpg
/content/drive/MyDrive/gan_det

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, Row_R1 to Row_R101
Columns: 137 entries, Column_1 to Column_137
dtypes: float64(137)
memory usage: 108.9+ KB


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, Row_F1 to Row_F101
Columns: 137 entries, Column_1 to Column_137
dtypes: float64(137)
memory usage: 108.9+ KB


In [ ]:
df1['prediction']=1
df2['prediction']=0

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, Row_R1 to Row_R101
Columns: 138 entries, Column_1 to prediction
dtypes: float64(137), int64(1)
memory usage: 109.7+ KB


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, Row_F1 to Row_F101
Columns: 138 entries, Column_1 to prediction
dtypes: float64(137), int64(1)
memory usage: 109.7+ KB


In [ ]:
df1.head(4)

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_129,Column_130,Column_131,Column_132,Column_133,Column_134,Column_135,Column_136,Column_137,prediction
Row_R1,191.0,464.0,197.0,548.0,210.0,626.0,229.0,703.0,266.0,771.0,...,597.0,756.0,550.0,769.0,520.0,775.0,489.0,772.0,0.235714,1
Row_R2,147.0,477.0,151.0,562.0,164.0,648.0,188.0,732.0,233.0,803.0,...,585.0,772.0,552.0,779.0,527.0,786.0,500.0,785.0,0.042857,1
Row_R3,236.0,465.0,241.0,545.0,250.0,623.0,258.0,699.0,281.0,771.0,...,631.0,751.0,537.0,785.0,503.0,789.0,469.0,782.0,0.357895,1
Row_R4,236.0,465.0,241.0,545.0,250.0,623.0,258.0,699.0,281.0,771.0,...,631.0,751.0,537.0,785.0,503.0,789.0,469.0,782.0,0.357895,1


In [ ]:
df1.tail(3)

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_129,Column_130,Column_131,Column_132,Column_133,Column_134,Column_135,Column_136,Column_137,prediction
Row_R498,119.0,491.0,125.0,583.0,134.0,673.0,152.0,760.0,200.0,837.0,...,590.0,768.0,565.0,768.0,542.0,773.0,516.0,771.0,0.065527,1
Row_R499,226.0,525.0,229.0,594.0,236.0,663.0,250.0,735.0,278.0,801.0,...,631.0,731.0,544.0,783.0,507.0,787.0,469.0,783.0,0.202703,1
Row_R500,226.0,525.0,229.0,594.0,236.0,663.0,250.0,735.0,278.0,801.0,...,631.0,731.0,544.0,783.0,507.0,787.0,469.0,783.0,0.202703,1


In [ ]:
df2.head(4)

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_129,Column_130,Column_131,Column_132,Column_133,Column_134,Column_135,Column_136,Column_137,prediction
Row_F1,274.0,495.0,275.0,564.0,286.0,632.0,305.0,699.0,326.0,767.0,...,597.0,768.0,516.0,778.0,487.0,779.0,463.0,775.0,0.515152,0
Row_F2,213.0,481.0,219.0,558.0,231.0,636.0,248.0,712.0,281.0,780.0,...,607.0,721.0,553.0,794.0,518.0,801.0,481.0,797.0,0.011364,0
Row_F3,295.0,496.0,285.0,558.0,284.0,620.0,305.0,685.0,334.0,750.0,...,650.0,749.0,514.0,807.0,475.0,803.0,442.0,791.0,0.125253,0
Row_F4,330.0,526.0,319.0,581.0,321.0,642.0,334.0,704.0,347.0,770.0,...,606.0,773.0,503.0,802.0,468.0,803.0,437.0,799.0,0.636364,0


In [ ]:
frames = [df1,df1,df1,df1,df1, df2,df2,df2,df2,df2]

df = pd.concat(frames)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1010 entries, Row_R1 to Row_F101
Columns: 138 entries, Column_1 to prediction
dtypes: float64(137), int64(1)
memory usage: 1.1+ MB


In [ ]:
df.head(5)

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_129,Column_130,Column_131,Column_132,Column_133,Column_134,Column_135,Column_136,Column_137,prediction
Row_R1,230.0,484.0,246.0,559.0,262.0,627.0,278.0,697.0,304.0,761.0,...,584.0,733.0,529.0,780.0,506.0,784.0,483.0,782.0,0.563218,1
Row_R2,194.0,521.0,197.0,602.0,204.0,682.0,221.0,763.0,257.0,835.0,...,623.0,744.0,569.0,768.0,536.0,771.0,500.0,770.0,0.213873,1
Row_R3,308.0,571.0,291.0,637.0,285.0,710.0,296.0,782.0,318.0,855.0,...,616.0,767.0,511.0,776.0,476.0,780.0,443.0,781.0,0.000000,1
Row_R4,190.0,495.0,193.0,579.0,204.0,663.0,213.0,745.0,236.0,823.0,...,636.0,753.0,560.0,800.0,523.0,805.0,484.0,803.0,0.000000,1
Row_R5,190.0,495.0,193.0,579.0,204.0,663.0,213.0,745.0,236.0,823.0,...,636.0,753.0,560.0,800.0,523.0,805.0,484.0,803.0,0.000000,1


In [ ]:
#df['prediction']=0
"""
import random
lst=random.sample(range(1,6), 4)
for x in lst :
  df.at['Row_' + str(x) , 'prediction'] = 1
"""

#lst
#df.at['Row_3', 'prediction'] = 1

"\nimport random\nlst=random.sample(range(1,6), 4)\nfor x in lst :\n  df.at['Row_' + str(x) , 'prediction'] = 1\n"

In [ ]:
df.head(5)

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_129,Column_130,Column_131,Column_132,Column_133,Column_134,Column_135,Column_136,Column_137,prediction
Row_R1,230.0,484.0,246.0,559.0,262.0,627.0,278.0,697.0,304.0,761.0,...,584.0,733.0,529.0,780.0,506.0,784.0,483.0,782.0,0.563218,1
Row_R2,194.0,521.0,197.0,602.0,204.0,682.0,221.0,763.0,257.0,835.0,...,623.0,744.0,569.0,768.0,536.0,771.0,500.0,770.0,0.213873,1
Row_R3,308.0,571.0,291.0,637.0,285.0,710.0,296.0,782.0,318.0,855.0,...,616.0,767.0,511.0,776.0,476.0,780.0,443.0,781.0,0.000000,1
Row_R4,190.0,495.0,193.0,579.0,204.0,663.0,213.0,745.0,236.0,823.0,...,636.0,753.0,560.0,800.0,523.0,805.0,484.0,803.0,0.000000,1
Row_R5,190.0,495.0,193.0,579.0,204.0,663.0,213.0,745.0,236.0,823.0,...,636.0,753.0,560.0,800.0,523.0,805.0,484.0,803.0,0.000000,1


In [ ]:
df.describe()

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_129,Column_130,Column_131,Column_132,Column_133,Column_134,Column_135,Column_136,Column_137,prediction
count,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,...,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000
mean,239.371287,493.688119,241.311881,566.782178,249.722772,639.608911,263.158416,712.227723,288.039604,780.722772,...,608.861386,752.153465,538.925743,774.292079,507.455446,777.782178,476.613861,773.801980,0.271428,0.500000
std,48.708044,27.021020,44.830114,24.648210,42.817199,25.151287,42.461603,27.578119,38.477574,29.855489,...,20.720199,15.675450,20.879555,20.569417,19.412851,20.732667,17.591032,19.887702,0.227893,0.500248
min,129.000000,411.000000,135.000000,495.000000,137.000000,570.000000,156.000000,641.000000,204.000000,703.000000,...,556.000000,715.000000,474.000000,724.000000,446.000000,724.000000,426.000000,723.000000,0.000000,0.000000
25%,204.000000,478.000000,206.000000,552.000000,216.000000,624.000000,231.000000,693.000000,259.000000,759.000000,...,594.000000,740.000000,529.000000,763.000000,498.000000,767.000000,466.000000,762.000000,0.037037,0.000000
50%,238.000000,495.000000,245.000000,565.000000,252.000000,636.000000,263.000000,709.000000,285.000000,777.500000,...,610.000000,753.500000,543.000000,775.000000,511.000000,779.000000,479.000000,775.000000,0.246884,0.500000
75%,269.000000,508.000000,269.000000,579.000000,277.000000,655.000000,287.000000,729.000000,306.000000,799.000000,...,625.000000,766.000000,553.000000,786.000000,519.000000,790.000000,489.000000,785.000000,0.457831,1.000000
max,382.000000,578.000000,372.000000,641.000000,372.000000,710.000000,389.000000,786.000000,409.000000,860.000000,...,651.000000,782.000000,579.000000,828.000000,548.000000,828.000000,526.000000,826.000000,0.900000,1.000000


In [ ]:

#df_test=df[(df == 0).all(axis=1)]
#df_test.info()


In [ ]:
cols=df.columns

for c in cols:
  print(df[c].isnull().sum())

#df['Column_1'].isnull().sum()

In [ ]:
DF=df

#Drop prediction Column

X = DF.drop(['prediction'], axis=1)
y = DF['prediction']

#shuffle rows
DF = DF.sample(frac = 1)

#show top 3 rows
DF.head(3)
#show last 3 rows
DF.tail(3)

# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


# check the shape of X_train and X_test
print("Train, Test data shape")
print(X_train.shape, X_test.shape)


cols = X_train.columns
#
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



Train, Test data shape
(808, 137) (202, 137)


In [ ]:
X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])

In [ ]:
# import SVC classifier
from sklearn.svm import SVC


# import metrics to compute accuracy
from sklearn.metrics import accuracy_score


# instantiate classifier with default hyperparameters
svc=SVC()


# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_test)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:

# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with default hyperparameters: 0.9901
